In [1]:
!pip install datasets
!pip install peft
!pip install bitsandbytes
!pip install transformers[torch]
!pip install pandas
!pip install trl
!pip install accelerate
!pip install google
!pip install tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 2.3 MB/s eta 0:00:00


In [2]:
from random import randrange
import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
    AutoModel,
)
from peft import LoraConfig
from trl import SFTTrainer
from sklearn.model_selection import train_test_split
import requests
from datasets import load_dataset

2024-04-30 12:59:25.305328: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-30 12:59:25.305456: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-30 12:59:25.438111: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
dataset = load_dataset("mahfoos/Patient-Doctor-Conversation")
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Description', 'Doctor', 'Patient', 'Status'],
        num_rows: 3325
    })
})


In [5]:
df = pd.DataFrame(dataset["train"])
df = df.drop("Status",axis=1)
df.head(5)

,Description,Doctor,Patient
0,what does abutment of the nerve root mean,hi I have gone through your query with diligen...,hi doctor I am just wondering what is abutting...
1,every time I eat spicy food I poop blood why,hello I have gone through your information and...,hi doctor I am a 26 year old male I am feet an...
2,will nano leo give permanent solution for erec...,hi for further doubts consult a sexologist online,hello doctor I am 48 years old I am experienci...
3,will kalarchikai cure multiple ovarian cysts i...,hello I just read your query see kalarachi kai...,hello doctor I have multiple small cysts in bo...
4,I masturbate only by rubbing the tip of the pe...,hi for further doubts consult a sexologist online,hi doctor during masturbation I just rub the t...


In [6]:
print("Length of dataset: " ,len(df))

Length of dataset:  3325


In [7]:
train_df = df.loc[:1999,:]
train_df.head(5)

,Description,Doctor,Patient
0,what does abutment of the nerve root mean,hi I have gone through your query with diligen...,hi doctor I am just wondering what is abutting...
1,every time I eat spicy food I poop blood why,hello I have gone through your information and...,hi doctor I am a 26 year old male I am feet an...
2,will nano leo give permanent solution for erec...,hi for further doubts consult a sexologist online,hello doctor I am 48 years old I am experienci...
3,will kalarchikai cure multiple ovarian cysts i...,hello I just read your query see kalarachi kai...,hello doctor I have multiple small cysts in bo...
4,I masturbate only by rubbing the tip of the pe...,hi for further doubts consult a sexologist online,hi doctor during masturbation I just rub the t...


In [8]:
test_df = df.loc[2000:,:]
test_df.head(5)

,Description,Doctor,Patient
2000,though I stopped using tobacco my gum turns bl...,hello it is good that you stopped chewing toba...,hello doctor I was chewing tobacco for the pas...
2001,do enlarged prostate of am suggest prostatitis,hello I have seen all your reports I think you...,hi doctor I am a 52 year old man my height is ...
2002,please advise medication for fever body pain a...,hello it is probably a viral infection along w...,hello doctor I am having slight fever at eveni...
2003,what causes frequent cramps with vaginal bleeding,hi overall looks like normal there is no ultra...,hello doctor after two weeks of lap I had mild...
2004,my dad has undergone angio for cad how many st...,hello I passed carefully through your dad medi...,hi doctor my dad recently had chest pain we to...


In [9]:
conversation_train = Dataset.from_pandas(train_df)
conversation_test = Dataset.from_pandas(test_df)

In [10]:
print(conversation_train)
print(conversation_test)

Dataset({
    features: ['Description', 'Doctor', 'Patient'],
    num_rows: 2000
})
Dataset({
    features: ['Description', 'Doctor', 'Patient'],
    num_rows: 1325
})


In [11]:
def format_prompt(sample):
    return f"""
Below is a DoctorInput and PatientInput that describes a conversation. Write a Description that appropriately describes the conversation.

###DoctorInput
{sample["Doctor"]}

###PatientInput
{sample["Patient"]}

###Description
{sample["Description"]}
"""

In [12]:
model_name = "meta-llama/Meta-Llama-3-8B"
use_flash_attention = False

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type ="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [13]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    use_cache = False,
    use_flash_attention_2 = use_flash_attention,
    device_map="auto",
    torch_dtype=torch.float16,
)

model.config.pretraining_tp=1

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM",
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="/llama-3",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="no",
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="linear",
    report_to="tensorboard",
)

In [16]:
from trl import SFTTrainer

max_seq_length = 512
trainer=SFTTrainer(
    model=model,
    train_dataset=conversation_train,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_prompt,
    args=args,
)

Generating train split: 0 examples [00:00, ? examples/s]

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [17]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,3.117700
20,2.968000
30,2.766400
40,2.603000
50,2.585300
60,2.476900
70,2.419900
80,2.463300
90,2.393200
100,2.455200


TrainOutput(global_step=873, training_loss=2.3368773389381503, metrics={'train_runtime': 13814.234, 'train_samples_per_second': 0.253, 'train_steps_per_second': 0.063, 'total_flos': 8.054506673248666e+16, 'train_loss': 2.3368773389381503, 'epoch': 3.0})

In [19]:
model_path = "./final_model"
model.save_pretrained(model_path)
# tokenizer.save_pretrained(model_path)

In [20]:
from huggingface_hub import notebook_login
notebook_login()

In [21]:
trainer.model.push_to_hub(repo_id="saadliaquat19/Llama-3-Patient-Doctor")

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/saadliaquat19/Llama-3-Patient-Doctor/commit/ef56c4292686f4f2a34498f014af55131f18027e', commit_message='Upload model', commit_description='', oid='ef56c4292686f4f2a34498f014af55131f18027e', pr_url=None, pr_revision=None, pr_num=None)

In [37]:
model_name = "meta-llama/Meta-Llama-3-8B"
use_flash_attention = False

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type ="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
from transformers import AutoModel
model = AutoModel.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    use_cache = False,
    use_flash_attention_2 = False,
    torch_dtype=torch.float16,
)

In [ ]:
from peft import PeftModel

adapter_model = PeftModel.from_pretrained(model, "saadliaquat19/Llama-3-Patient-Doctor")

In [ ]:
model2 = adapter_model.merge_and_unload()

In [ ]:
model2.push_to_hub(repo_id="saadliaquat19/Llama-3-Patient-Doctor")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.push_to_hub("saadliaquat19/Llama-3-Patient-Doctor")